In [ ]:
konjungsi = ['dan', 'atau']

In [178]:
def gabung2AMRmenjadi1(kumpulan_amr,kata_sambung):    
    if penman.__version__ != '1.1.0':
        package = 'Penman==1.1.0'
        pip.main(['install', package])
        reload(penman)

    g_per_kalimat = [] 

    #nama variablenya diubah menjadi vv_ dulu 
    for indeks, amr in enumerate(kumpulan_amr) : 
        graph = penman.decode(amr)
        t = penman.configure(graph)
        t.reset_variables('vv{i}')
        graph = penman.interpret(t)

        g_per_kalimat.append(penman.encode(graph))

    penambahan = 1 
    
    temp_root = '(vv0 / '+kata_sambung+')'
    g = penman.decode(temp_root)

    #digitnya vv_ asli akan ditambah sesuai dengan jumlah variabel amr graph kalimat sebelumnya
    for indeks, amr in enumerate(g_per_kalimat) : 
        graph = penman.decode(amr)
        max_vv = 0
        
        for x,y in enumerate(graph.triples):
            old_vv = y[0][2:] #dari
            new_vv = int(old_vv)+penambahan #dari
            var = 'vv{0}'.format(new_vv)
            old_vv_ = 0
            
            rel = y[1]
            
            if rel==':instance':
                con = y[2]
            else:
                try: 
                    old_vv_ = y[2][2:] #ke
                    new_vv_ = int(old_vv_)+penambahan #ke
                    con = 'vv{0}'.format(new_vv_)
                except:
                    con = y[2]
            
            if x == 0:
                g.triples.extend([(tuple(('vv0',':op'+str(indeks+1),var)))])
            
            g.triples.extend([(tuple((var,rel,con)))])
            
            try:
                if int(old_vv) > max_vv:
                    max_vv = int(old_vv)
                if int(old_vv_) > max_vv:
                    max_vv = int(old_vv_)
            except:
                pass
                
            if x == len(graph.triples)-1:
                if max_vv > len(graph.instances()):
                    penambahan = penambahan + max_vv
                else:
                    penambahan = penambahan + len(graph.instances())
    
    #nama variable diubah menjadi inisial_ kembali
    t = penman.configure(g)
    t.reset_variables('{prefix}{j}')
    g = penman.interpret(t)
    graph = penman.encode(g) 
    
    return graph

# Kalimat Yang Mau Digabung

In [151]:
data_awal = 'amr_simple_amany.txt'
my_data = {}
with open(data_awal) as file:
    amr=''
    for line in file:
        if '\n' == line:
            if id_amr in my_data:
                if sent not in my_data[id_amr]['sent']:
                    my_data[id_amr]['sent'].append(sent)
                if amr not in my_data[id_amr]['amr']:
                    my_data[id_amr]['amr'].append(amr)                
            else:
                my_data[id_amr] = {'sent':[sent],'amr':[amr]}
            amr=''
            continue
        if ('# ::id ' in line):
            id_amr = line.split()[2]
#             print(id_amr, type(id_amr))
        elif ('# ::snt ' in line):
            temp_sent = line.split()[2:]
            sent = ' '.join(temp_sent)
#             print(sent)
        else:
            amr+=line
#             print(amr)

# Buat AMR gabungan dan Kalimat gabungan

In [195]:
count = 1
list_key = list(my_data.keys())
id_amr_terakhir = list_key[-1] #700
list_kalimat = []
list_amr = []

for i,j in enumerate(my_data.keys()):
    if i%2 ==0: #setiap genap
        id1 = list_key[i] #genap
        id2 = list_key[i+1] #ganjil
        
        sent1 = my_data[id1]['sent'][0]
        sent2 = my_data[id2]['sent'][0]
        
        
        amr1 = my_data[id1]['amr'][0]
        amr2 = my_data[id2]['amr'][0]
        
        kumpulan_amr = [amr1,amr2]
        kata_sambung = 'dan'
        graf_gabungan = gabung2AMRmenjadi1(kumpulan_amr,kata_sambung)

#         if id1 == '8':
#             print(amr1)
#             print(amr2)
#             print(graf_gabungan)

        list_kalimat.append(sent1+' '+kata_sambung+' '+sent2)
        list_amr.append(graf_gabungan)
        count+=1
        
for i,j in enumerate(my_data.keys()):
    if i%2 ==0: #setiap genap
        id1 = list_key[i] #genap
        id2 = list_key[i+1] #ganjil
        
        sent1 = my_data[id1]['sent'][0]
        sent2 = my_data[id2]['sent'][0]
        
        
        amr1 = my_data[id1]['amr'][0]
        amr2 = my_data[id2]['amr'][0]
        
        kumpulan_amr = [amr1,amr2]
        kata_sambung = 'atau'
        graf_gabungan = gabung2AMRmenjadi1(kumpulan_amr,kata_sambung)

#         if id1 == '8':
#             print(amr1)
#             print(amr2)
#             print(graf_gabungan)

        list_kalimat.append(sent1+' '+kata_sambung+' '+sent2)
        list_amr.append(graf_gabungan)
        count+=1

# Masukkan ke file baru

In [196]:
data_akhir = 'amr_simple_amany_v2.txt'
with open(data_akhir, 'w', encoding='utf-8') as file:
    file.write('')    
for i,j in enumerate(list_kalimat):
    with open(data_akhir, 'a', encoding='utf-8') as file:
        file.write('# ::id '+str(i+1+int(id_amr_terakhir))+'\n')
        file.write('# ::snt '+list_kalimat[i]+'\n')
        file.write(list_amr[i]+'\n\n')

In [197]:
len(list_kalimat)

524

In [198]:
len(my_data.keys())

524

In [182]:
i =90
print(list_kalimat[i])
print(list_amr[i])

Mereka akhirnya memilih untuk bergabung dan Pemandangannya terlihat indah
(d / dan
   :op1 (p / pilih
           :ARG0 (m / mereka
                    :ARG1 (g / gabung)))
   :op2 (l / lihat
           :ARG1 (p2 / pandang
                     :mod (i / indah))))
